In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from train import *

In [ ]:
from infer import *
import notebook_utils as nutils
import cv2
import os
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

# import sys;
# sys.path.append('/home/haotruong/Receipt/EasyOCR')

# import easyocr
# reader = easyocr.Reader(['en'], gpu=True)

def recognize_text(img, bboxes, target=None):
    offset = 5

    total_result = []
    
    for bbox in np.array(bboxes, dtype=np.int32)[:]:
        x0, y0, x1, y1 = bbox
        i = img[y0 - offset: y1+offset, x0-offset : x1+offset]

        results = []

        for n in range(4):
            data = reader.recognize(np.rot90(i, n))
            text = data[0][1]
            if target:
                if (target in text) or (text in target):
                    results.append(text)
            else:
                results.append(data[0][1])
            
        total_result.append(results)

    return total_result

classes = ['ocr']
classes = ['552F', '550SF','550PF', '552SF', '551F', '551GPF', '522F']
predictor = get_predictor("ocr_spec.pth", classes, 800)
file = 'OCR/552F/11.JPG'
bboxes, classes, scores, v = infer('ocr', predictor, file, True, 0.5)

In [ ]:
base_classes = ['552F', '550SF','550PF', '552SF', '551F', '551GPF', '522F']

In [ ]:
files = nutils.list_files("OCR", "**/*.JPG")

In [ ]:
mp4_files = [Path("ocr_video.mp4")]

In [ ]:
a = np.array([])
len(np.where(a > 0)[0])

In [ ]:
for path in mp4_files:
    out = cv2.VideoWriter("temp.mp4",cv2.VideoWriter_fourcc(*'MP4V'), 10, (640, 360)) #(640, 360) (285, 360)
    
    vc = cv2.VideoCapture(str(path))
    time_per_frame = (1/vc.get(cv2.CAP_PROP_FPS)) * 1000

#     vc.set(1, 2400);

    rval = True

    frame_id = 0
    max_frame = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
#     max_frame=200
    
    counter = 0

    circle_coord = [200, 200]
    pbar = tqdm(total=max_frame, position=0, leave=True)
    while rval:
        rval , frame = vc.read()

        if rval:
            start = time.time()

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            bboxes, classes, scores, v = infer('baocam', predictor, image, True, 0.5)
            
            if len(classes) > 0:
                catch_len = len(np.where(classes > 0)[0])
                if catch_len > 0:
                    ###False
                    v.draw_circle(circle_coord, 'b', radius=50)
                else:
                    v.draw_circle(circle_coord, 'g', radius=50)
            else:
                v.draw_circle(circle_coord, 'y', radius=50)


            
            total_time = (time.time() - start) * 1000

            for bbox, class_id in zip(bboxes, classes):
                v.draw_box(bbox, alpha = 1)
                v.draw_text(base_classes[class_id], (int((bbox[0] + bbox[2])/2) , int((bbox[1])) - 20), font_size= 20, color='b')

            predicted_image = v.output.get_image()

            skip_frame = int(np.ceil(total_time / time_per_frame))
            skip_frame = 3
#             print(skip_frame)
            out.write(predicted_image)

            frame_id += 1
            pbar.update(1)
            
            for i in range(skip_frame):
                out.write(predicted_image)
                vc.read()
                pbar.update(1)
                frame_id += 1

            if frame_id > (max_frame - 0):
                break

    out.release()
    vc.release()

    new_file = "{}_export.mp4".format(path.stem).replace(" ", "") 
    cmd = "ffmpeg -i temp.mp4 -vcodec libx264 {} -y".format(new_file)
    print(cmd)
    os.system(cmd)

In [ ]:
from IPython.display import Video

Video("ocr_video_export.mp4", width = 800)


In [ ]:
for index, file in enumerate(files):    
    try:
        bboxes, classes, scores, v = infer('ocr', predictor, str(file), True, 0.5, True)

        target = parse_file(str(file.with_suffix(".xml")))['object'][0]['name']

    #         text_list = recognize_text(v.img, bboxes, target=target)

        for classid, bbox in zip(classes, bboxes):
            text = base_classes[classid]
            v.draw_text(text, (int((bbox[0] + bbox[2])/2) , int((bbox[1])) - 20), font_size= 20, color='r')

        cv2.imwrite("OCR/OCR_Result_2/{}.jpg".format(index), cv2.cvtColor(v.output.get_image(), cv2.COLOR_RGB2BGR )   )

        print("{} / {}".format(index+1, len(files)))
    except:
        pass
#     nutils.imshow(v.output.get_image(), 55)

In [ ]:
files = nutils.list_files("OCR/OCR", "*.JPG")
for i, f in enumerate(files):
    bboxes, classes, scores, v = infer('ocr', predictor, str(f), True, 0.5)
    for bbox in bboxes:
        v.draw_box(bbox, alpha = 1)

    predicted_image = v.output.get_image()
    
    cv2.imwrite("OCR/OCR_Result/{}.jpg".format(i), cv2.cvtColor(predicted_image, cv2.COLOR_RGB2BGR) )
#     nutils.imshow(predicted_image)
#     plt.show()


In [ ]:
import notebook_utils as nutils


In [ ]:
nutils.imshow(cv2.imread("OCR/OCR_Result_2/{}.jpg".format(300)), 10)

In [ ]:
mp4_files = nutils.list_files("data", "CAM 1-*.mp4")
path = mp4_files[0]

In [ ]:
time_per_frame = (1/30) * 1000

In [ ]:
for path in mp4_files:
    out = cv2.VideoWriter("temp.mp4",cv2.VideoWriter_fourcc(*'MP4V'), 30, (640, 360))
    vc = cv2.VideoCapture(str(path))

#     start_frame = 10800#12600 #10800
#     vc.set(1,start_frame);

    rval = True

    frame_id = 0
    max_frame = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

    counter = 0

    pbar = tqdm(total=max_frame, position=0, leave=True)
    
    mot_tracker = Sort(min_hits= 0)
    mot_tracker.reset_count()
    while rval:
        rval , frame = vc.read()

        if rval:
            start = time.time()

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            bboxes, classes, scores, v = infer('baocam', predictor, image, True, 0.5)
            track_bbs_ids = mot_tracker.update(bboxes)

            for bbox in bboxes:
                v.draw_box(bbox, alpha = 1)

            for bbox in track_bbs_ids:
                counter = bbox[-1]
                v.draw_text(str(int(counter)), (int( (bbox[0] + bbox[2]) /2 ) , int((bbox[1] + bbox[3]) /2 )), font_size= 70, color='g')


            v.draw_text(str(int(counter)), (200 , 200), font_size= 200, color='g')

            predicted_image = v.output.get_image()
            total_time = (time.time() - start) * 1000

            skip_frame = int(np.ceil(total_time / time_per_frame))

            out.write(predicted_image)

            frame_id += 1
            pbar.update(1)
            
            for i in range(skip_frame):
                out.write(predicted_image)
                vc.read()
                pbar.update(1)
                frame_id += 1


    #         plt.imshow(predicted_image)
    #         plt.show()

#             print(frame_id)
            if frame_id > (max_frame - 100):
                break

    out.release()
    vc.release()

    new_file = "{}_export.mp4".format(path.stem).replace(" ", "") 
    cmd = "ffmpeg -i temp.mp4 -vcodec libx264 {} -y".format(new_file)
    print(cmd)
    os.system(cmd)

In [ ]:
from IPython.display import Video

Video(new_file, width = 800)

In [ ]:
out.release()
vc.release()

In [ ]:
v.draw_box(bboxes[0])

out = v.output.get_image()

nutils.imshow(out, 2)

In [ ]:
vc = cv2.VideoCapture(str(path))

start_frame = 12640
vc.set(1,start_frame);

mot_tracker = Sort(min_hits= 0) 
mot_tracker.reset_count()
counter = 0

In [ ]:
rval , frame = vc.read()
start = time.time()

image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
bboxes, classes, scores, v = infer('baocam', predictor, image, True, 0.5)

track_bbs_ids = mot_tracker.update(bboxes)


for bbox in bboxes:
    v.draw_box(bbox, 1)
    
for bbox in track_bbs_ids:
    counter = bbox[-1]    
    v.draw_box(bbox[:4], edge_color='r')

        
v.draw_text(str(int(counter)), (200 , 200), font_size= 200, color='g')

predicted_image = v.output.get_image()

total_time = (time.time() - start) * 1000

skip_frame = int(np.ceil(total_time / time_per_frame))
for i in range(skip_frame):
    vc.read()
    
nutils.imshow(predicted_image, 15)